# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto-2
[ IBM Data Science Professional Certificate](https://www.coursera.org/professional-certificates/ibm-data-science)<br>
Applied Data Science Capstone - Week3 <br><br>
Coded and Maintained by <b>Mohammed Burhanuddeen R</b><br>
Github account [@Metallurgist](https://github.com/Metallurgist/Doom-Slayer)

This notebook addresses only <b>Question 2</b> as requested by the Assignment</b><br>
Click the link to see [Question 1](https://github.com/Metallurgist/Metallurgist-Peer-graded-Assignment-Segmenting-and-Clustering-Neighborhoods-in-Toronto/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto-1.ipynb)<br>
Click the link to see [Full Assignment](https://github.com/Metallurgist/Metallurgist-Peer-graded-Assignment-Segmenting-and-Clustering-Neighborhoods-in-Toronto/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto_Full.ipynb)<br>

<b>Question 2 :</b><br>
Get the latitude and the longitude coordinates of each neighborhood and read into a Pandas DataFrame.

<b>Solution :</b><br>
In order to get the co-ordinates [Geocoder Python package](https://geocoder.readthedocs.io/index.html) can be used.<br>
The user can use the following codes to install and use the <i>Geocoder Python Package<i>

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

But since the package is very unrealiable and the user should be very persistent to make the calls.<br>
There is a limit on how many times you can call geocoder.google function.<br>
It is <b>2500 times per day</b><br><br>
<b>Hence this [.csv file]( http://cocl.us/Geospatial_data) containing the geographical coordinates of each postal code is used to keep this notebook simple<b>

In [1]:
!pip install BeautifulSoup4

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [2]:
!pip install lxml
!pip install et_xmlfile

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

In [4]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names = ['Postalcode','Borough','Neighborhood']
df_source= pd.DataFrame(columns = column_names)

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df_source.loc[len(df_source)] = row_data

In [5]:
df_clean= df_source[df_source.Borough != 'Not assigned']

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df_clean.groupby(['Postalcode', 'Borough'])
df = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

print("Source DataFrame Shape : ",df_source.shape)
print("DataFrame after Cleaning : ",df.shape)

df.head(10)

Source DataFrame Shape :  (180, 3)
DataFrame after Cleaning :  (103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<hr><b>Reading and joining the Geo-csv file with the cleaned df Dataframe</b>

In [6]:
csv_file="https://cocl.us/Geospatial_data"
geo=pd.read_csv(csv_file)

print("Size of Geo-csv file :",geo.shape)
geo.head()

Size of Geo-csv file : (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
geo.rename(columns={"Postal Code":"Postalcode"},inplace=True)
geo_merge=pd.merge(geo,df, on="Postalcode")

geo_df=geo_merge[["Postalcode","Borough","Neighborhood","Latitude","Longitude"]]

print("Mergerd Dataframe Size :",geo.shape)
geo_df.head()

Mergerd Dataframe Size : (103, 3)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<hr>